In [1]:
import tensorflow as tf
import numpy as np
import os


In [2]:
#in interactive mode, there is no __file__ defined
#os.getcwd() is a workaround
modelpath=os.path.dirname(__file__) if('__file__') in dir() else os.getcwd()

In [3]:
def read_csv(batch_size,file_name,record_defaults):
    filename_queue = tf.train.string_input_producer ( 
        [os.path.join(modelpath,file_name)])
    reader = tf.TextLineReader(skip_header_lines=1)
    key,value=reader.read(filename_queue)
    #decode-csv will convert a Tensor from type string (the text line) 
    #in a tuple of tensor columns with the specified defaults, 
    #which also sets the data type for each column
    decoded=tf.decode_csv(value,record_defaults=record_defaults)
    
    #batch actually reads the file and loads "batch_size" rows in a single tensor
    
    return tf.train.shuffle_batch(decoded,batch_size=batch_size,
                                 capacity=batch_size*50, 
                                 min_after_dequeue=batch_size)
    

In [4]:
#demo of logistic to answer Yes/No question
#initialize variables/model parameters
W=tf.Variable(tf.zeros([4,3]),dtype=tf.float32,name="weights")
b = tf.Variable(tf.zeros([3],name="bias",dtype=tf.float32))

In [5]:
def inputs():
    batch_size=100
    sepal_length,sepal_width,petal_length,petal_width,label = \
     read_csv(batch_size,'irisData/iris.data.csv', [[0.0], [0.0], [0.0], [0.0], [""]])
    #convert class names to a 0 based class index
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))
    
    #pack all features in a single matrix
    #transpose to a matrix with one example per row
    #and each feature dimension per column
    features = tf.transpose(tf.stack(
    [sepal_length,sepal_width,petal_length,petal_width]
    ))
    
    return features, label_number

In [6]:
def combine_inputs(X):
    return tf.matmul(X,W)+b
#define the training loop operations
def inference(X):
    #compute the inference model over data X and return the result
    #return tf.sigmoid(combine_inputs(X))
    return tf.nn.softmax(combine_inputs(X))
def loss(X,Y):
    #compute loss over training data X and expected output Y
    Y_predicted = inference(X)
    
    #applying cross entropy reduction
    err=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X),labels=Y))
    return err

def train(total_loss):
    #train/adjust model parameters according to computed total loss
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    return optimizer

In [7]:
def evaluate(sess,X,Y):
    #evaluate the resulting trained model
    #softmax output a row matrix, axis=1
    pre=inference(X)
    predicted = tf.cast( tf.argmax(  inference(X) ,1),tf.int32 )
    result=sess.run(tf.reduce_mean(tf.cast (tf.equal(predicted,Y),tf.float32)))
    #print("softmax dim:{0}".format(tf.shape(pre)))
    print("result:{0}".format(result))
    return

In [8]:
#Create a saver
saver = tf.train.Saver()

In [9]:
#Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X,Y = inputs()
    total_loss = loss(X,Y)
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    
    initial_step=0
    
    #verify if we don't have a checkpoint saved already
    ckpt = tf.train.get_checkpoint_state(modelpath)
    
    if ckpt and ckpt.model_checkpoint_path:
        #Restores from checkpoints
        saver.restore(sess,ckpt.model_checkpoint_path)
        initial_step = int(ckpt.model_checkpoint_path.rsplit('-',1)[1])
    
    #actual training loop
    training_steps=2000
    
    for step in range(initial_step,training_steps):
        sess.run([train_op])
        if step%1000 ==0:
            saver.save(sess,"my-model",global_step=step)
        #for debugging and learning purposes, see how the loss gets decremented through training steps
        if step%10 ==0:
            print("loss: {0}".format(sess.run([total_loss])))
            evaluate(sess,X,Y)
            
    coord.request_stop()
    coord.join(threads)
    saver.save(sess,"my-model",global_step=training_steps)
    sess.close()

loss: [1.0829422]
result:0.23000000417232513
loss: [1.0066336]
result:0.2800000011920929
loss: [0.9433763]
result:0.6899999976158142
loss: [0.92135805]
result:0.6700000166893005
loss: [0.86434525]
result:0.6499999761581421
loss: [0.84150803]
result:0.6200000047683716
loss: [0.82750201]
result:0.6399999856948853
loss: [0.76475394]
result:0.5799999833106995
loss: [0.73945451]
result:0.6399999856948853
loss: [0.7242856]
result:0.6399999856948853
loss: [0.68752921]
result:0.8199999928474426
loss: [0.65934944]
result:0.7200000286102295
loss: [0.67731839]
result:0.6899999976158142
loss: [0.66148996]
result:0.7400000095367432
loss: [0.6401729]
result:0.8100000023841858
loss: [0.59644526]
result:0.8399999737739563
loss: [0.63686907]
result:0.7699999809265137
loss: [0.59551054]
result:0.7900000214576721
loss: [0.62661016]
result:0.7300000190734863
loss: [0.61088258]
result:0.7900000214576721
loss: [0.62469274]
result:0.7599999904632568
loss: [0.57684147]
result:0.699999988079071
loss: [0.594420

loss: [0.24934933]
result:0.9700000286102295
loss: [0.28822753]
result:0.9800000190734863
loss: [0.28183395]
result:0.9700000286102295
loss: [0.26692352]
result:0.9700000286102295
loss: [0.30759612]
result:0.949999988079071
loss: [0.29890025]
result:0.949999988079071
loss: [0.31370434]
result:0.9700000286102295
loss: [0.28445008]
result:0.949999988079071
loss: [0.27329475]
result:0.9599999785423279
loss: [0.25155216]
result:0.9800000190734863
loss: [0.23778538]
result:0.9700000286102295
loss: [0.25329819]
result:0.9599999785423279
loss: [0.27154523]
result:0.9800000190734863
loss: [0.25404984]
result:0.9700000286102295
loss: [0.25456774]
result:0.949999988079071
